# Network Traffic Analysis Pipeline

## 0. Setup Environment

0.1. Install all needed dependencies


In [ ]:
%pip install pyshark
%pip install nest_asyncio
%pip install pandas
%pip install pymongo
%pip install python-dotenv
%pip install scikit-learn
%pip install matplotlib

0.2. Allow the notebook to work assynchronously.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

0.3. Setup global variables

In [ ]:
import time
import sys
sys.path.append('..')
run_seed=int(time.time())
base_training_percentage = 60
online_training_percentage = 20
validation_percentage = 20

## 1. Data Gathering

1.1. Download datasets

1.2. Store datasets in a local folder

1.3. Select and label the datasets.
For each dataset, add it to the list of datasets in a tuple with the format `(dataset_path, dataset_label)`, where:
- `dataset_path` is the path to the dataset file
- `dataset_label` is the label that should be assigned to the entries, this can be either `normal`,`anomaly` or `unknown`(for unsupervised training)

In [ ]:
dataset_files = []

## 2. Data Preprocessing

2.1. Import the desired preprocessor.

In [ ]:
from src.preprocessors.pcap_preprocessor import PcapPreprocessor

preprocessor = PcapPreprocessor()

2.2. Load and preprocess the datasets.

    This step may depend on the preprocessor used. 
    The preprocessor should store the data and preprocess the data, providing methods to access it.
    In the case of classes that don't store the data in memory, this step can be skipped after the first execution. 


In [ ]:
if len(dataset_files) > 0:
    preprocessor.load_datasets(dataset_files)

## 3. Data Visualization

3.1. Gather all the data in a single dataframe for visualization.

In [ ]:
import pandas as pd
all_data = preprocessor.get_all_data()
all_data = preprocessor.preprocess_dataframe(all_data)

3.X. Add vizualization steps here

In [ ]:
print(all_data.info())
for col in all_data.columns: 
    print(col)

## 4. Model Training

4.1. Select and import the desired models.

In [ ]:
from src.models.offline.offline1 import Offline_RandomForest
from src.models.online.online1 import Online_RandomForest


model_list = [Offline_RandomForest(), Online_RandomForest()]

4.2. Gather the data for training.

When training online models a second dataframe is provided to simulate the online training.z

In [ ]:
training_data,online_training_data = preprocessor.get_training_data(base_training_percentage, online_training_percentage, True, seed=run_seed)

print(training_data.info())
print(online_training_data.info())

4.3. Train the models using the offline data.

In [ ]:
for model in model_list:
    model.train(training_data)

4.4. Train the online models simulating an online enviorment.

In [ ]:
for model in model_list:
    if model.is_online():
        for index, row in online_training_data.iterrows():
            model.predict(row)

## 5. Model Evaluation

5.1. Select and import the desired evaluator.

In [ ]:
from src.evaluators.standard_evaluator import Evaluator

evaluator = Evaluator()

5.2. Gather the data for evaluation.

In [ ]:
validation_data = preprocessor.get_validation_data(validation_percentage,seed=run_seed)

5.3. Get the predictions from all models into a dataframe.

In [ ]:
# Header
header = ['expected']
for model in model_list:
    header.append(model.id)
validation_results = [header]

# Results
labels = validation_data['label']
validation_data = validation_data.drop(columns=['label'])
model_results = []
for model in model_list:
    model_results.append(model.predict_batch(validation_data))

for i in range(len(labels)):
    row = [labels[i]]
    for model_result in model_results:
        row.append(model_result[i])
    validation_results.append(row)

df = pd.DataFrame(validation_results)
print(df)

5.4. Evaluate the models using their predictions.

In [ ]:
results = evaluator.evaluate(df)

5.5. Analyze the results.

In [ ]:
print(results)

In [ ]:
evaluator.model_comparison(results)

In [ ]:
evaluator.heatmap_performance_metrics(results)

In [ ]:
evaluator.roc_curve_per_model(df)